In [1]:
import requests
from pyquery import PyQuery as pq
from tqdm import tqdm

import gzip
import csv
import os
import glob
from collections import OrderedDict
import io
import codecs

In [7]:
datadir = "data/"
filepre = "Annual"
baseurl = "https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/"
filespec = "StormEvents_details-ftp_v1.0_d"
destination = "stormeventshistorical.csv"
tempfile = "goddamn_gzip_is_such_a_clusterfuck"

In [ ]:
html = requests.get(baseurl).content

In [ ]:
todownload = []
for link in pq(html)("a"):
    href = pq(link)("a").attr('href').strip()
    if filespec in href:
        todownload.append(href)

In [ ]:
for item in tqdm(todownload):
    filename = datadir + filepre + item.replace(filespec, "")
    remoteurl = baseurl + item
    with open(filename, "wb") as f:
        f.write(requests.get(remoteurl).content)

In [3]:
gzipfiles = list(glob.glob(datadir + filepre + "*.gz"))

In [8]:
masterlist = []
for gzipfile in tqdm(gzipfiles):
    # with gzip.open(gzipfile, mode='rt') as f:   # Goddamn non-conformant syntax.
    # with gzip.open(gzipfile, mode='rb') as f:   # Goddamn non-conformant syntax.
    with gzip.open(gzipfile, mode="rb") as f:
        with open(tempfile, "wb") as moarf:
            moarf.write(f.read())
        # textfilehandle = io.TextIOWrapper(f, buffer=50*1000*1000)
    with open(tempfile, "r") as textfilehandle:
        reader = csv.DictReader(textfilehandle)
        for row in reader:
            masterlist.append(row)

100%|██████████████████████████████████████████████████████████████████████████████████| 70/70 [05:53<00:00, 42.63s/it]


In [10]:
if os.path.exists(tempfile):
    os.remove(tempfile)

In [9]:
with open(destination, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(list(row.keys()))
    for row in masterlist:
        writer.writerow(list(row.values()))
    